# Introduction to Rivet Transpiler

**Quantum Transpilation** is the transformation of a given virtual quantum circuit:

- to match the topology of a specific device
- to optimize the circuit for execution


## 1. Transpilation Stages
**Pass Manager** is an internal Qiskit object constructed by the `transpile` function "under the hood" during transpilation.
Pass Manager consists of Passes which analyse or change the transpiled circuit.
Calling `pass_manager.run` is equivalent to calling `transpile` and passing a circuit, backend and corresponding transpilation parameters.

In [1]:
import qiskit

pass_manager = qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager(
    # backend=backend,
    # initial_layout=[1, 0, 2],
    optimization_level=3,
    seed_transpiler=1234
)

Calling `pass_manager.stages` the main transpilation stages can be displayed: ('init', 'layout', 'routing', 'translation', 'optimization', 'scheduling')

1) **Init** - Unrolling custom instructions and converting the circuit to all 1 and 2 qubit gates.

2) **Layout** - Mapping circuit virtual qubits to backend physical qubits.

3) **Routing** - Injecting SWAP gates to comply with the backend’s coupling map.

4) **Translation** - Translating to the target backend’s basis gate set.

5) **Optimization** - Main optimization loop to increase circuit quality.

6) **Scheduling** - Conversion to hardware-specific pulses.


#### **The Rivet Transpiler package** provides a family of functions (transpile and service functions) for the efficient transpilation of quantum circuits.
 

## 2. Rivet Transpiler: Transpile Functions

`transpile` - a custom transpilation with possibility of using additional parameters. Interface of the `transpile` function matches `qiskit.transpile` - but allows additional parameters like:

- pre-defined transpilation stacks
- custom PassManager
- dynamical decoupling
- transpiler options

Running `transpile` function with `return_metrics=True` additionally returns metrics of the transpilation process in the following form:

```
{'pass_index': 0,
 'pass_name': 'ContainsInstruction',
 'pass_type': 'Analysis',
 'time': 9.5367431640625e-06,
 'depth': 5,
 'width': 3,
 'ibm_cost': 0.9498972910479807,
 'gates_counter': {1: 3, 2: 3}}
 ...
```

`transpile_chain` - consistently transpiles and "stitches" a chain of quantum circuits.  For more details, check the [Circuit Stitching](circuit_stitching/circuit_stitching.ipynb) notebook.

`transpile_right` / `transpile_left`  - transpiles an additional circuit to the right/left part of the existing circuit. Basic transpilation is slow, `transpile_right` / `transpile_left` are much faster. Check the implementation for the case of the [Shadow State Tomography](shadows/shadow_state_tomography.ipynb).

`transpile_and_compress` - transpiles and "topologically compresses" a circuit considering a coupling map of the selected backend to avoid using unnecessary ancilla qubits. For more details, check the [Qubit-Constrained Transpilation](qubit_constrained_transpilation/qubit_constrained_transpilation.ipynb) notebook.



# 3. Rivet Transpiler: Service Functions  
In the Rivet Transpiler, the following service functions are defined:  

`get_full_map` - To get final qubit allocation after transpilation. Check the [Qubit-Constrained Transpilation](qubit_constrained_transpilation/qubit_constrained_transpilation.ipynb) nonebook for more details.

`get_circuit_hash` - calculates circuit hash. Check [Hashing](hashing/hashing.ipynb)  nonebook for more details.

`get_cnot_circuit` - creates a CNOT test circuit.
  
`get_litmus_circuit` -creates a 
**Litmus circuit** which is a minimal quantum circuit which consisting of:

- Parametrized RZ gate on each qubit (each parameter name matches the  index of the virtual qubit - to trace consequent permutations of qubits during layout and routing phases of transpilation).

- Circular CNOT gates (necessary to introduce SWAPs during transpilation to not fully connected topology).

`get_ibm_cost` -  IBM Cost is a product of gate fidelities* (F) and depth penalization** (K): $
C=K^d \prod_{j \in G}F_j
$

* K - circuit depth penalization factor
* d - circuit depth
* G - gates
* F - fidelities

IBM Cost can be used as a circuit quality metric (e.g. to compare the cost for the litmus and transpiled circuits) (see e.g. “Demonstration of quantum volume 64 on a superconducting quantum computing system” – IBMQ – P. Jurcevic, A. Javadi-Abhari, L. S. Bishop and others – https://arxiv.org/abs/2008.08571).

*the degree of accuracy with which a quantum gate operation is executed on a quantum computing system, measured as the agreement between the physical implementation and the ideal mathematical operation.

**technique used in quantum circuit optimization to reduce the depth of a quantum circuit.

## 4. Supported Stacks

Make sure you have followed and installed Rivet Transpiler. Check the installation steps from   **Readme.md**.
Then import Qiskit and other modules required for the examples and Rivet Transpiler functions execution.


Four pre-defined transpilation stacks have been added:

- Qiskit transpilation
- Pytket transpilation
- BQSKit QSearch synthesis
- BQSKit QFactor instantiation

### Stacks Usage

In [2]:
from rivet_transpiler import transpile
from rivet_transpiler import get_litmus_circuit

from qiskit_ibm_runtime.fake_provider import FakeLimaV2

backend = FakeLimaV2()

QUBITS_COUNT = 3

litmus_circuit = get_litmus_circuit(QUBITS_COUNT, "Litmus")

litmus_circuit.draw()

┌──────────────┐          ┌───┐ Litmus 
Litmus_0_0: ┤ Rz(Litmus_0) ├──■───────┤ X ├───░────
            ├──────────────┤┌─┴─┐     └─┬─┘   ░    
Litmus_0_1: ┤ Rz(Litmus_1) ├┤ X ├──■────┼─────░────
            ├──────────────┤└───┘┌─┴─┐  │     ░    
Litmus_0_2: ┤ Rz(Litmus_2) ├─────┤ X ├──■─────░────
            └──────────────┘     └───┘        ░

In [3]:
STACKS = ["qiskit",
          "qiskit_qsearch",
          "qiskit_qfactor_qsearch",
          "qiskit_pytket"]

#### Qiskit

In [4]:
transpiled_circuit = transpile(
    litmus_circuit,
    backend,
    stack="qiskit",
    seed_transpiler=1234)

transpiled_circuit.draw()

┌──────────────┐          ┌───┐          ┌───┐ Litmus 
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├───────■──┤ X ├──■───────┤ X ├───░────
                ├──────────────┤     ┌─┴─┐└─┬─┘┌─┴─┐┌───┐└─┬─┘   ░    
Litmus_0_0 -> 1 ┤ Rz(Litmus_0) ├──■──┤ X ├──■──┤ X ├┤ X ├──■─────░────
                ├──────────────┤┌─┴─┐└───┘     └───┘└─┬─┘        ░    
Litmus_0_1 -> 2 ┤ Rz(Litmus_1) ├┤ X ├─────────────────■──────────░────
                └──────────────┘└───┘                            ░    
 ancilla_0 -> 3 ──────────────────────────────────────────────────────
                                                                      
 ancilla_1 -> 4 ──────────────────────────────────────────────────────

#### Pytket

In [5]:
transpiled_circuit = transpile(
    litmus_circuit,
    backend,
    stack="qiskit_pytket",
    seed_transpiler=1234)

transpiled_circuit.draw()

┌──────────────┐          ┌───┐          ┌───┐ ░ 
Litmus_0_2 -> 0 ┤ Rz(Litmus_2) ├───────■──┤ X ├──■───────┤ X ├─░─
                ├──────────────┤     ┌─┴─┐└─┬─┘┌─┴─┐┌───┐└─┬─┘ ░ 
Litmus_0_0 -> 1 ┤ Rz(Litmus_0) ├──■──┤ X ├──■──┤ X ├┤ X ├──■───░─
                ├──────────────┤┌─┴─┐└───┘     └───┘└─┬─┘      ░ 
Litmus_0_1 -> 2 ┤ Rz(Litmus_1) ├┤ X ├─────────────────■────────░─
                └──────────────┘└───┘                          ░ 
 ancilla_0 -> 3 ─────────────────────────────────────────────────
                                                                 
 ancilla_1 -> 4 ─────────────────────────────────────────────────

## 5. Transpilation Examples

- [Shadow State Tomography](shadows/shadow_state_tomography.ipynb)
- [Fourier Adder](fourier_adder/fourier_adder.ipynb)
- [Qubit-Constrained Transpilation](qubit_constrained_transpilation/qubit_constrained_transpilation.ipynb)
- [Hashing](hashing/hashing.ipynb) 
- [Circuit Stitching](circuit_stitching/circuit_stitching.ipynb)
- [Quantum circuit synthesis](circuit_synthesis/circuit_synthesis.ipynb)
